### Дополнительное задание  

#### Реализовать алгоритм kNN самому, голыми руками. Реализацию построить в виде класса с методами fit(), predict(). Кодить в отдельном файле .py. Прислать сравнение качества своего алгоритма и алгоритма из sklearn.

#### Введение

Метод k-ближайших соседей используется для решения задачи классификации. Он относит объекты к классу, которому принадлежит большинство из k его ближайших соседей в многомерном пространстве признаков.
Число k – это количество соседних объектов в пространстве признаков, которые сравниваются с классифицируемым объектом. Иными словами, если k=10, то каждый объект сравнивается с 10-ю соседями. 
Выбор параметра k противоречив. С одной стороны, увеличение его значения повышает достоверность классификации, но при этом границы между классами становятся менее четкими. На практике хорошие результаты дают эвристические методы выбора параметра k, например, перекрестная проверка.
Главным недостатком метода является высокая вычислительная трудоемкость, которая увеличивается квадратично с ростом числа обучающих примеров.

kNN (k-identify nearest neighbors) – это т.н. ленивый классификатор.
Это означает, что в процессе обучения он не делает ничего, а только хранит тренировочные данные. Он начинает классификацию только тогда, когда появляются новые немаркированные данные.
Активный же классификатор создает классификационную модель в процессе обучения. Когда вводятся новые данные, такой классификатор «скармливает» данные классификационной модели.

Когда появляется новые неразмеченные данные, kNN проходит по 2 базовым шагам:
1.Сначала он ищет k ближайших размеченных точек данных – другими словами, k ближайших соседей.
2.Затем, используя классы соседей, kNN решает, как лучше классифицировать новые данные.

Для непрерывных данных kNN использует дистанционную метрику, например, Евклидову дистанцию (метрику).

При работе с дискретными данными, они сначала преобразуются в непрерывные. Вот 2 примера:
Использование расстояния Хэмминга как метрики для определения «близости» двух текстовых строк;
Преобразование дискретных данных в бинарные числа.

Как kNN решает, к какому классу отнести данные, если соседи не принадлежат одному классу?
Для решения этой проблемы используются 2 классические техники:
1. Принять за правильное решение простое большинство. К какому классу относится наиболее количество соседей, туда и определяют точку данных.
2. Проделать то же самое, но дать ближайшим соседям больший вес. Самый простой способ сделать это – использовать квантиль расстояния. Если сосед отстоит на 5 единиц, то его вес будет 1/5. При увеличении дистанции вес становится все меньше и меньше. Это как раз то, что нам нужно.

Вот 5 вещей, за которыми нужно следить:
1. kNN может быть очень ресурсозатратным, если пытаться определить ближайших соседей на большом наборе данных.
2. Зашумленные данные могут испортить kNN-классификацию.
3. Нужно учитывать количество значений. Характеристики с большим количеством значений могут оказывать влияние на дистанционную метрику, по отношению к характеристикам с меньшим количеством значений.
4. Поскольку обработка данных «откладывается», kNN обычно требует больше места, чем активные классификаторы.
5. Выбор правильной дистанционной метрики очень важен для точности kNN.

Расстояние между точками с заданными координатами в n-мерном пространстве находится по формуле d = sqrt((a1 - b1)2 + (a2 - b2)2 + ... + (an - bn)2), где sqrt - квадратный корень, а1...an - координаты первой точки, b1..bn - соответствующие координаты второй точки.

#### Датасет для тестирования 

In [100]:
import pandas as pd
import numpy as np

In [101]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/thomas/anaconda2/envs/env_py2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


##### Для отладки алгоритма и сравнения результатов берем учебную выборку с kaggle

Задача на kaggle: https://www.kaggle.com/c/bioresponse
Данные: https://www.kaggle.com/c/bioresponse/data
По данным характеристикам молекулы требуется определить, будет ли дан биологический ответ (biological response). Признаки нормализованы.

In [102]:
bioresponce = pd.read_csv('bioresponse.csv', header=0, sep=',')
bioresponce.head(1)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.0,0.497009,0.1,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0


In [103]:
# формируем вектор целевых значений y
# и матрицу признаков X
y = bioresponce.Activity.values
X = bioresponce.iloc[:, 1:]

In [104]:
# вызываем метод train_test_split для разделения выборки на обучающую и целевую
from sklearn.model_selection import train_test_split
# отдельно указываем, что тестовая подвыборка = 25% от всей выборки
(X_train, 
 X_test, 
 y_train, 
 y_test) = train_test_split(X, y, test_size=0.25, random_state=0)
# проверяем результат разделения
print X.shape, X_train.shape, X_test.shape
print y.shape, y_train.shape, y_test.shape

(3751, 1776) (2813, 1776) (938, 1776)
(3751,) (2813,) (938,)


#### Стандартный алгоритм

In [39]:
# проводим проверку качества стандартного инструмента sklearn.neighbors - KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
n_neighbor = [3, 5, 7, 9]
for n_neighbors in n_neighbor:
    weight = ['uniform', 'distance']
    for weights in weight: 
        ps = [1, 2]
        for p in ps:
            neigh = KNeighborsClassifier(n_neighbors, weights, p=p)
            neigh.fit(X_train, y_train) 
            predict = neigh.predict(X_test)
            right_prediction = 0

            for i in range(len(predict)):
                if predict[i] == y_test[i]:
                    right_prediction += 1
                else:
                    pass
            print 'n_neighbors=', n_neighbors, 'weights=', weights, 'p=', p, 'KNC accuracy ', float(right_prediction)/y_test.shape[0]
# исследуем результат работы аглоритма при разных параметрах

n_neighbors= 3 weights= uniform p= 1 KNC accuracy  0.753731343284
n_neighbors= 3 weights= uniform p= 2 KNC accuracy  0.745202558635
n_neighbors= 3 weights= distance p= 1 KNC accuracy  0.757995735608
n_neighbors= 3 weights= distance p= 2 KNC accuracy  0.749466950959
n_neighbors= 5 weights= uniform p= 1 KNC accuracy  0.746268656716
n_neighbors= 5 weights= uniform p= 2 KNC accuracy  0.736673773987
n_neighbors= 5 weights= distance p= 1 KNC accuracy  0.757995735608
n_neighbors= 5 weights= distance p= 2 KNC accuracy  0.750533049041
n_neighbors= 7 weights= uniform p= 1 KNC accuracy  0.740938166311
n_neighbors= 7 weights= uniform p= 2 KNC accuracy  0.735607675906
n_neighbors= 7 weights= distance p= 1 KNC accuracy  0.752665245203
n_neighbors= 7 weights= distance p= 2 KNC accuracy  0.751599147122
n_neighbors= 9 weights= uniform p= 1 KNC accuracy  0.713219616205
n_neighbors= 9 weights= uniform p= 2 KNC accuracy  0.716417910448
n_neighbors= 9 weights= distance p= 1 KNC accuracy  0.738805970149
n_n

Наблюдаем наиболее высокое значение точности 0.757995735608
при применении количества ближних - 3 и 5
и введении весового параметра distance
а также расчете расстояния по манхеттенской метрике.
Принимаем указанное значение как целевое. 

#### Собственный алгоритм

In [105]:
import pandas as pd
import numpy as np

In [106]:
bioresponce = pd.read_csv('bioresponse.csv', header=0, sep=',')

In [240]:
y = bioresponce.Activity.values
X = bioresponce.iloc[:, 1:]

from sklearn.model_selection import train_test_split

(X_train, 
 X_test, 
 y_train, 
 y_test) = train_test_split(X, y, test_size=0.25, random_state=0)

In [301]:
from collections import defaultdict, Counter
import operator

class KNN:
    # это метод инициализации объекта данного класса, куда мы передаем
    # количество ближайших, метод расчета весов при них(с учетом удаленности) или без весов,
    # и способ расчета расстояния (эвклидов или манхеттенский)
    # по дефолту считаем по 5 бижайшим соседям, веса не применяем, 
    # метод расчета расстояний - Эвклидов
    def __init__(self, n_neighbors=5, weights='uniform', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.p = p
    # поскольку это т.н. ленивый классификатор, то в процессе обучения он не делает ничего, 
    # а только хранит тренировочные данные. Он начинает классификацию только тогда, когда 
    # появляются новые немаркированные данные.  
    def fit(self, X_train, y_train):
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)
    # метод предсказывает принадлежность к классу      
    def predict(self, X_test):
        p = self.p
        n_neighbors = self.n_neighbors
        weights = self.weights
        X_train = self.X_train
        y_train = self.y_train
        X_test = np.array(X_test)
        pediction_vector = []
        for i in range(len(X_test)): 
            # отладочный
            print i
            dictance_between_objects_array = []
            
            # расчет Эвклидовой метрики
            if p == 2:
                for j in range(len(X_train)):
                    e_dist = np.linalg.norm(X_test[i] - X_train[j])
                    # формируем массив из дистанций до классифицируемого объекта
                    dictance_between_objects_array.append(e_dist)
                
            # расчет Манхэттенской метрики        
            elif p == 1:
                for j in range(len(np.array(X_train))):
                    m_dist = map(lambda x,y: math.fabs(float(x)-float(y)),
                                 np.array(X_test)[i],np.array(X_train)[j]) 
                    m_dist = sum(m_dist)/100.
                    # формируем массив из дистанций до классифицируемого объекта
                    dictance_between_objects_array.append(m_dist)
                           
            # задаем словарь из пар "расстояние"-"класс"
            neighbors_dictionary = {}
            for i, j in zip(dictance_between_objects_array, y_train):
                neighbors_dictionary.setdefault(i,[]).append(j)
            # сортируем ключи-расстояния в порядке возрастания и формируем из них массив
            sorted_distances = sorted(neighbors_dictionary.keys())
            # технический массив
            self.sorted_distances = sorted_distances
            
            neighbors_list = []
            if weights == 'uniform':
            # формируем массив из n_neighbors классов с наименьшими дистанциями до классифицируемого 
            # объекта
                for g in range(n_neighbors):
                    neighbors_list.append(neighbors_dictionary.get(sorted_distances[g]))
                    
                neighbor = []
                for h in neighbors_list:    
                    neighbor.append(h[0])            
                # считаем количество объектов разных классов
                counter = Counter(neighbor)
                prediction = max(counter.iteritems(), key=operator.itemgetter(1))[0]
                # формируем вектор прогнозов класса объектов тестовой выборки 
                pediction_vector.append(prediction)
                self.pediction_vector = pediction_vector    
            
                  
            # формируем массив из n_neighbors классов с учетом удаленности
            elif weights == 'distance':
                short_neighbors_dictionary = {}
                short_key = []
                short_values = []
                # веса считаем как частное: 1/(порядковый номер по отношению к целевому объекту)
                for g in range(n_neighbors):
                    short_key.append(1/float(g+1))
                    short_values.append(neighbors_dictionary.get(sorted_distances[g])[0])
                # получаем словарь где ключи это классы а значения - массив весов
                for i, j in zip(short_key, short_values):
                    short_neighbors_dictionary.setdefault(j,[]).append(i) 
                # 
                sum_dict = {}
                sum_key = []
                sum_value = []
                for i in range(len(short_neighbors_dictionary.items())):
                    sum_value.append(short_neighbors_dictionary.items()[i][0])
                    sum_key.append(sum(short_neighbors_dictionary.items()[i][1]))
                for i, j in zip(sum_key, sum_value):
                    sum_dict.setdefault(i,[]).append(j)
                #print sum_dict   
                sorted_weights = sorted(sum_dict.items())
                #print sorted_weights[-1][1]
                pediction_vector.append(sorted_weights[-1][1])
                self.pediction_vector = pediction_vector        
        
    # метод считает точность предсказания 
    def accuracy(self, y_test):
        pediction_vector = self.pediction_vector
        right_prediction = 0
        for i in range(len(pediction_vector)):
            if pediction_vector[i] == y_test[i]:
                right_prediction += 1
            else:
                pass
    
        print 'KNN accuracy ', float(right_prediction)/len(pediction_vector)
        

### результат РАВЕН результату стандартного алгоритма

In [233]:
test_base = KNN()
test_base.fit(X_train, y_train)
test_base.predict(X_test)
test_base.accuracy(y_test)
# точность алгоритма при базовых настройках, то есть при 
# 5 соседях, без применения весов, и расчете дистанции по Эвклидовой метрике
# n_neighbors= 5 weights= uniform p= 2 KNC accuracy  0.736673773987

KNN accuracy  0.736673773987


### результат РАВЕН результату стандартного алгоритма

In [171]:
test_uniform = KNN(n_neighbors=3, weights='uniform', p=2)
test_uniform.fit(X_train, y_train)
test_uniform.predict(X_test)
test_uniform.accuracy(y_test)
# точность алгоритма при 3 соседях, без применения весов, и расчете дистанции
# по Эвклидовой метрике
# n_neighbors= 3 weights= uniform p= 2 KNC accuracy  0.745202558635

KNN accuracy  0.745202558635


### результат ВЫШЕ результата стандартного алгоритма

In [224]:
test_distance = KNN(n_neighbors=5, weights='distance', p=2)
test_distance.fit(X_train, y_train)
test_distance.predict(X_test)
test_distance.accuracy(y_test)
# точность алгоритма при 5 соседях, применении весов, и расчете дистанции
# по Эвклидовой метрике выше аналогичного расчета при использовании стандартного модуля
# n_neighbors= 5 weights= distance p= 2 KNC accuracy  0.750533049041

KNN accuracy  0.761194029851


In [320]:
(X_train_50, 
 X_test_50, 
 y_train_50, 
 y_test_50) = train_test_split(X, y, test_size=0.50, random_state=0)

In [321]:
(X_train_25, 
 X_test_25, 
 y_train_25, 
 y_test_25) = train_test_split(X, y, test_size=0.25, random_state=0)

In [322]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, weights='distance', p=1)
neigh.fit(X_train_50, y_train_50) 
predict = neigh.predict(X_test_25)
right_prediction = 0

for i in range(len(predict)):
    if predict[i] == y_test[i]:
        right_prediction += 1
    else:
        pass
print 'KNC accuracy ', float(right_prediction)/y_test.shape[0]
# исследуем результат работы аглоритма при разных параметрах

KNC accuracy  0.726012793177


### результат НИЖЕ результата стандартного алгоритма

In [324]:
test_distance_man = KNN(n_neighbors=5, weights='distance', p=1)
test_distance_man.fit(X_train_50, y_train_50)
test_distance_man.predict(X_test_25)
test_distance_man.accuracy(y_test)
# точность алгоритма при 5 соседях, применении весов, и расчете дистанции
# по Манхэттенской метрике

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

#### Выводы из работы

1. Написанный самостоятельно алгоритм классификации по методу ближайших соседей 
проверен на учебном наборе данных bioresponse.csv.
2. Проведено сравнение точности классификации алгоритма с точностью классификации 
стандартного модуля библиотеки sklearn.neighbors.KNeighborsClassifier.
3. Проверкой и сравнением установлено: 
3.1. Точность классификации при базовых настройках идентична.
3.2. Точность классификации при применении весовых коэффициентов выше у самостоятельно
написанного алгоритма.
3.3. Точность классификации при применении Манхэттенской метрики ниже у самостоятельно
написанного алгоритма. 
3.4. Скорость обработки данных выше у стандартного модуля.

Вывод: алгоритм, работающий с точностью, сопоставимой со стандартной, создан, что свидетельствует об общем понимании принципов работы изучаемого алгоритма. 
